In [ ]:
import os

import numpy as np
import pandas as pd
from PIL import Image

from matplotlib import pyplot as plt
from matplotlib.path import Path

def add_labels_to_mask(pustule_mask, labels):
    if len(labels_dataframe['x'])==0: 
        print('Label file is empty.')
        return pustule_mask
    
    if bool(labels_dataframe['x'].iloc[0]=='None')==True:
        print('First row is \'None\' implying no pusutles.')
        # This convention was used to signify if a leaf was labeled, but
        # conatined no pustules
        return pustule_mask
    
    # If both checks passed, then loop over rows in labels
    for row_number in np.arange(len(labels))[:]:
        
        # Grab the x/y coordinates of the edge of the labeled elipse
        x_points = np.fromstring(labels['x'].iloc[row_number], sep=',', dtype='int')
        y_points = np.fromstring(labels['y'].iloc[row_number], sep=',', dtype='int')
        
        # Use matplotlib Path function to create elipse from points
        label_elipse = Path(np.vstack((x_points,y_points)).T)
        
        # Get max row/col extent of label
        min_x = x_points.min()-1
        max_x = x_points.max()+1
        min_y = y_points.min()-1
        max_y = y_points.max()+1

        # Get candidate coordinates that might fall within elipse
        xx, yy = np.meshgrid(np.arange(min_x, max_x), np.arange(min_y, max_y))
        all_x_points = xx.ravel()
        all_y_points = yy.ravel()

        # Check if elipse contains these points
        mask = label_elipse.contains_points(np.vstack((all_x_points,all_y_points)).T, radius=0)
        
        # Mask for points within or on the edge of the elipse
        all_x_points = all_x_points[mask]
        all_y_points = all_y_points[mask]

        # If points fall outside image (occasionally where label is right on edge),
        # then set to max row/col
        all_x_points[all_x_points>(num_cols-1)] = num_cols-1
        all_y_points[all_y_points>(num_rows-1)] = num_rows-1

        # Set coordinates in full mask array to 1 (indicating pustule)
        pustule_mask[all_y_points, all_x_points] = 1
        
    return pustule_mask

### Inputs ###

# filename of .tif file (like 20181117_0003_14.tif)
image_path = '/mnt/nvme2/cwhite/ml_leaf_tutorial/training/full_tifs/20181117_0003_14.tif' 

# filename of corresponding annotation file (should have same name, but end in .csv)
csv_path = '/mnt/nvme2/cwhite/ml_leaf_tutorial/training/annotations/20181117_0003_14.csv' 

save_figure = True # Fallse if you do not want to save a plot
save_figure_path = '' # path to where you want this plot saved, anything if above is False
###############

# Read the labels from the .df
labels_dataframe = pd.read_csv(csv_path)

# Open the image and convert to numpy array
img = Image.open(image_path)
img = np.array(img,dtype='float32')

#Create empy pusutle mask the same size as the image
num_rows, num_cols = img.shape[0], img.shape[1]
pustule_mask = np.zeros([num_rows,num_cols])

# Call the function above to insert the annotations from labels_dataframe 
# into the 2-D pustule_mask array
pustule_mask = add_labels_to_mask(pustule_mask, labels_dataframe)


# If specified, then plot the leaf image, pusutle mask, and overlay
if save_figure==True:
    plt.figure(figsize=(10,4))

    plt.subplot(311)
    plt.imshow(img/255.0, interpolation='nearest')
    plt.xticks([]);
    plt.yticks([]);

    plt.subplot(312)
    plt.imshow(pustule_mask, interpolation='nearest')
    plt.xticks([]);
    plt.yticks([]);

    plt.subplot(313)
    plt.imshow(img/255.0, interpolation='nearest')
    plt.imshow(pustule_mask, interpolation='nearest', alpha=0.25)
    plt.xticks([]);
    plt.yticks([]);
    plt.tight_layout()
    plt.savefig(save_figure_path + os.path.basename(image_path)[0:-4] + '.png', dpi=100)